# Predicting House Sale Prices

## Imports

In [1]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

# Functions

In [16]:
def drop_above_cutoff_null_columns(df, cutoff):
    null_stats = df.isnull().sum()
    above_cutoff_cols = null_stats[(null_stats >= df.shape[0] * cutoff)].index
    return df.drop(above_cutoff_cols, axis=1)

def drop_above_cutoff_nulls_in_text_columns(df, cutoff=1):
    null_stats = df.select_dtypes(include=["object"]).isnull().sum().sort_values(ascending=False)
    above_cutoff_text_cols = null_stats[null_stats > 0]
    return df.drop(above_cutoff_text_cols, axis=1)

def fill_numerical_values(df):
    null_stats = df.select_dtypes(include=["int", "float"]).isnull().sum().sort_values(ascending=False)


def transform_features(df, cutoff=0.25):
    df = drop_above_cutoff_null_columns(df, cutoff)
    df = drop_above_cutoff_nulls_in_text_columns(df, 1)
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]

    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])

    features = numeric_train.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])

    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    return np.sqrt(mse)


In [3]:
df = pd.read_csv("AmesHousing.tsv", sep="\t")

In [4]:
transformed_df = transform_features(df)
filtered_df = select_features(transformed_df)

In [5]:
rmse = train_and_test(filtered_df)
print(rmse)

57088.25161263909


In [24]:
null_stats = df.select_dtypes(include=["int", "float"]).isnull().sum().sort_values(ascending=False)

In [28]:
null_stats

Lot Frontage       490
Garage Yr Blt      159
Mas Vnr Area        23
Bsmt Half Bath       2
Bsmt Full Bath       2
BsmtFin SF 1         1
Garage Cars          1
Garage Area          1
Total Bsmt SF        1
Bsmt Unf SF          1
BsmtFin SF 2         1
Kitchen AbvGr        0
3Ssn Porch           0
Screen Porch         0
Enclosed Porch       0
Open Porch SF        0
Wood Deck SF         0
Pool Area            0
Misc Val             0
Mo Sold              0
Yr Sold              0
Fireplaces           0
TotRms AbvGrd        0
Order                0
Bedroom AbvGr        0
Half Bath            0
Full Bath            0
PID                  0
Gr Liv Area          0
Low Qual Fin SF      0
2nd Flr SF           0
1st Flr SF           0
Year Remod/Add       0
Year Built           0
Overall Cond         0
Overall Qual         0
Lot Area             0
MS SubClass          0
SalePrice            0
dtype: int64